https://gist.github.com/qpwo/c538c6f73727e254fdc7fab81024f6e1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import count
from collections import namedtuple
import os
import json
import pandas as pd
import random
%matplotlib inline

In [2]:
!make build

Building Cython extension
python setup.py build_ext --inplace
running build_ext


In [3]:
from mcts import MCTS, KlondikeNode
from benchmarking import random_state
random.seed(0)
root_state = KlondikeNode(*random_state())
tree = MCTS()
state = root_state

In [4]:
visited = set([ state ])
for i in count(1):
    print()
    print(i, state)
    print(state.to_pretty_string())
    for _ in range(5):
        tree.do_rollout(state)
    state = tree.choose(state)
    if state in visited:
        raise Exception(f'cycle: {state} was already visited')
    visited.add(state)
    if state.is_terminal():
        print("TERMINAL")
        print(state.to_pretty_string())
        break


1 K98879
Stock: TC 8D 6D 7D 7H AH 2D 2H 4H 2S 2C 7C KH JC AD 6C 3H 7S 3C 6S AS 3S 5H TD
Waste: 
Fnd C: 
Fnd D: 
Fnd S: 
Fnd H: 
Tab 1: KS
Tab 2: 4d 9H
Tab 3: th ac 4C
Tab 4: 8h qc js 5S
Tab 5: 5d qh kc jh 8S
Tab 6: qd 3d 4s 9d qs 5C
Tab 7: kd 9s 6h jd 9c 8c TS

2 K4985
Stock: TC 8D 6D 7D 7H AH 2D 2H 4H 2S 2C 7C KH JC AD 6C 3H 7S 3C 6S AS
Waste: TD 5H 3S
Fnd C: 
Fnd D: 
Fnd S: 
Fnd H: 
Tab 1: KS
Tab 2: 4d 9H
Tab 3: th ac 4C
Tab 4: 8h qc js 5S
Tab 5: 5d qh kc jh 8S
Tab 6: qd 3d 4s 9d qs 5C
Tab 7: kd 9s 6h jd 9c 8c TS

3 K73521
Stock: TC 8D 6D 7D 7H AH 2D 2H 4H
Waste: TD 5H 3S AS 6S 3C 7S 3H 6C AD JC KH 7C 2C 2S
Fnd C: 
Fnd D: 
Fnd S: 
Fnd H: 
Tab 1: KS
Tab 2: 4d 9H
Tab 3: th ac 4C
Tab 4: 8h qc js 5S
Tab 5: 5d qh kc jh 8S
Tab 6: qd 3d 4s 9d qs 5C
Tab 7: kd 9s 6h jd 9c 8c TS

4 K72685
Stock: TC 8D 6D 7D 7H AH 2D 2H 4H
Waste: TD 5H 3S AS 6S 3C 7S 3H 6C AD JC KH 7C 2C 2S
Fnd C: 
Fnd D: 
Fnd S: 
Fnd H: 
Tab 1: KS
Tab 2: 4D
Tab 3: th ac 4C
Tab 4: 8h qc js 5S
Tab 5: 5d qh kc jh 8S
Tab 6: qd 3d

Exception: cycle: K9245 was already visited

In [9]:
vis = {str(s): s for s in visited}

In [40]:
tree.children[vis['K43231']]

{K94074, K9245, K45953, K1354, K23002, K85924, K75700, K76025}

In [39]:
tree.choose(vis['K23259'])

K71577

In [ ]:
# ch = tree.choose(state)
# print(ch)
# ch in visited

In [6]:
print(state)
print(tree.children[state])

def score(n):
    if tree.N[n] == 0:
        return float("-inf")  # avoid unseen moves
    return tree.Q[n] / tree.N[n]  # average reward

for c in tree.children[state]:
    print(c, score(c), tree.Q[c], tree.N[c])

K9245
{K23259, K74501, K39557, K87079, K10526, K59507, K85077, K71577}
K23259 -0.1 -0.9 9
K74501 -0.1 -0.8 8
K39557 -0.1 -0.5 5
K87079 -0.1 -0.4 4
K10526 -0.1 -0.4 4
K59507 -0.1 -0.3 3
K85077 -0.1 -0.8 8
K71577 -0.1 -0.4 4


In [ ]:
k25, k27, k29 = tree._select(ch)

In [ ]:
children = list(tree.children[state])
children

In [ ]:
tree.Q[state]

In [ ]:
tree.N[state]

In [ ]:
state_backup

In [ ]:
# state_backup = state

In [ ]:
path = []
node = state
while True:
    if node in path:
        raise ValueError("detected cycle")
    path.append(node)
    print(path)
    if node not in tree.children or not tree.children[node]:
        # node is either unexplored or terminal
        print("return path", path)
        break
    unexplored = tree.children[node] - tree.children.keys()
    if unexplored:
        n = unexplored.pop()
        path.append(n)
        print("return path", path)
        break
    node = tree._uct_select(node)  # descend a layer deeper
    print('selected', node)

In [ ]:
state.find_children()

In [ ]:
print(state.to_pretty_string())

In [ ]:
tree.do_rollout(state)

In [ ]:
random.seed(0)
root_state = KlondikeNode(*random_state())
tree = MCTS()
state = root_state

import cProfile
cProfile.run('for _ in range(5): tree.do_rollout(state)', 'restats')

import pstats
from pstats import SortKey
p = pstats.Stats('restats')
p.sort_stats(SortKey.CUMULATIVE).print_stats()

import os
os.remove('restats')